<a href="https://colab.research.google.com/github/SahandShabanloueii/ML/blob/main/ID3-DecisionTree.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Understanding the ID3 Algorithm for Decision Trees

The ID3 (Iterative Dichotomiser 3) algorithm is a classic method for constructing decision trees. Developed by Ross Quinlan in 1986, ID3 is widely used for both classification and regression tasks. Let's dive into the details:

## How Does ID3 Work?

1. **Entropy and Information Gain**:
   - ID3 aims to split the dataset into subsets that are as homogeneous as possible with respect to the target variable (class labels).
   - It starts at the root node and selects the attribute that provides the most information gain.
   - **Entropy**: A measure of impurity or randomness in the dataset. Lower entropy indicates more order.
   - **Information Gain**: The reduction in entropy achieved by splitting on a specific attribute.

2. **Recursive Splitting**:
   - At each node, ID3 selects the attribute with the highest information gain.
   - It recursively splits the dataset into subsets based on the chosen attribute values.
   - The process continues until a stopping criterion is met (e.g., all instances belong to the same class or no more features to split on).

3. **Tree Structure**:
   - The resulting decision tree has internal nodes representing attributes and leaf nodes representing class labels.
   - Each internal node tests an attribute, and each branch corresponds to a possible attribute value.
   - The tree is built top-down, greedily selecting the best attribute at each step.

4. **Pruning (Optional)**:
   - To prevent overfitting, post-pruning techniques can be applied.
   - Pruning involves removing branches that do not significantly improve accuracy on validation data.

## Advantages of ID3:

- **Interpretability**: The resulting decision tree is easy to visualize and explain.
- **Simplicity**: ID3 is straightforward to implement.
- **Handles Categorical Data**: ID3 works well with categorical features.

## Limitations:

- **Sensitive to Noisy Data**: ID3 can overfit if the dataset contains noise.
- **Biased Toward Attributes with Many Values**: Attributes with more values tend to have higher information gain.

Note that: While ID3 is a foundational algorithm, more advanced variants like C4.5 and CART that are interduced later for DecisionTree modellings, address some of its limitations.


## Import necessary libraries

In [1]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd

## Loading the Iris dataset to fit and test our model later

In [2]:
# Load the Iris dataset
iris = load_iris()
X = iris.data
y = iris.target
feature_names = iris.feature_names

# Split the dataset into a training set and a testing set
# test_size=0.2 means 20% of the data will be used for testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Converting the Sklearn dataset to a DataFrame
*italicized text*

In [3]:
train_data = pd.DataFrame(X_train, columns=feature_names)
train_data['class'] = y_train

test_data = pd.DataFrame(X_test, columns=feature_names)
test_data['class'] = y_test

In [4]:
train_data

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),class
0,4.6,3.6,1.0,0.2,0
1,5.7,4.4,1.5,0.4,0
2,6.7,3.1,4.4,1.4,1
3,4.8,3.4,1.6,0.2,0
4,4.4,3.2,1.3,0.2,0
...,...,...,...,...,...
115,6.1,2.8,4.0,1.3,1
116,4.9,2.5,4.5,1.7,2
117,5.8,4.0,1.2,0.2,0
118,5.8,2.6,4.0,1.2,1


In [5]:
test_data

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),class
0,6.1,2.8,4.7,1.2,1
1,5.7,3.8,1.7,0.3,0
2,7.7,2.6,6.9,2.3,2
3,6.0,2.9,4.5,1.5,1
4,6.8,2.8,4.8,1.4,1
5,5.4,3.4,1.5,0.4,0
6,5.6,2.9,3.6,1.3,1
7,6.9,3.1,5.1,2.3,2
8,6.2,2.2,4.5,1.5,1
9,5.8,2.7,3.9,1.2,1


## Defining the entropy of a dataset:

In [6]:
def entropy(target_col):
    elements, counts = np.unique(target_col, return_counts=True)
    entropy = np.sum([(-counts[i]/np.sum(counts)) * np.log2(counts[i]/np.sum(counts)) for i in range(len(elements))])
    return entropy

## Defining the Information Gain of an attribute:

In [7]:
def InfoGain(data, split_attribute_name, target_name="class"):
    # Calculate the entropy of the total dataset
    total_entropy = entropy(data[target_name])

    # Calculate the values and the corresponding counts for the split attribute
    vals, counts = np.unique(data[split_attribute_name], return_counts=True)

    # Calculate the weighted entropy
    Weighted_Entropy = np.sum([(counts[i]/np.sum(counts)) * entropy(data.where(data[split_attribute_name]==vals[i]).dropna()[target_name]) for i in range(len(vals))])

    # Calculate the information gain
    Information_Gain = total_entropy - Weighted_Entropy
    return Information_Gain

## And finally defining the ID3 algorithm:

In [8]:
def ID3(data, originaldata, features, target_attribute_name="class", parent_node_class=None):
    # Define the stopping criteria --> If one of this is satisfied, we want to return a leaf node#

    # If all target_values have the same value, return this value
    if len(np.unique(data[target_attribute_name])) <= 1:
        return np.unique(data[target_attribute_name])[0]

    # If the dataset is empty, return the mode target feature value in the original dataset
    elif len(data) == 0:
        return np.unique(originaldata[target_attribute_name])[np.argmax(np.unique(originaldata[target_attribute_name], return_counts=True)[1])]

    # If the feature space is empty, return the mode target feature value of the direct parent node
    elif len(features) == 0:
        return parent_node_class

    # If none of the above conditions holds true, grow the tree!

    else:
        # Set the default value for this node --> The mode target feature value of the current node
        parent_node_class = np.unique(data[target_attribute_name])[np.argmax(np.unique(data[target_attribute_name], return_counts=True)[1])]

        # Select the feature which best splits the dataset
        item_values = [InfoGain(data, feature, target_attribute_name) for feature in features] # Return the information gain values for the features in the dataset
        best_feature_index = np.argmax(item_values)
        best_feature = features[best_feature_index]

        # Create the tree structure. The root gets the name of the feature with the maximum information gain
        tree = {best_feature:{}}

        # Remove the feature with the best information gain from the feature space
        features = [i for i in features if i != best_feature]

        # Grow a branch under the root node for each possible value of the root node feature

        for value in np.unique(data[best_feature]):
            value = value
            # Split the dataset along the value of the feature with the largest information gain and thereby create sub_datasets
            sub_data = data.where(data[best_feature] == value).dropna()

            # Call the ID3 algorithm for each of those sub_datasets with the new parameters
            subtree = ID3(sub_data, originaldata, features, target_attribute_name, parent_node_class)

            # Add the sub tree, grown from the sub_dataset to the tree under the root node
            tree[best_feature][value] = subtree

        return tree

## Function to predict for any input sample

In [9]:
def predict(query, tree, default=1):
    for key in list(query.keys()):
        if key in list(tree.keys()):
            try:
                result = tree[key][query[key]]
            except:
                return default

            result = tree[key][query[key]]
            if isinstance(result, dict):
                return predict(query, result)
            else:
                return result

## Function to test the prediction accuracy of the tree

In [10]:
def test(data, tree):
    queries = data.iloc[:,:-1].to_dict(orient="records")
    predicted = pd.DataFrame(columns=["predicted"])

    # Calculate the prediction accuracy
    for i in range(len(data)):
        predicted.loc[i,"predicted"] = predict(queries[i], tree, 1.0)
    print('The prediction accuracy is: ',(np.sum(predicted["predicted"] == data["class"])/len(data))*100,'%')

## Create the decision tree using the ID3 algorithm

In [ ]:
decision_tree = ID3(train_data, train_data, train_data.columns[:-1])

## Test the decision tree's accuracy on the test data

In [11]:
test(test_data, decision_tree)

The prediction accuracy is:  83.33333333333334 %
